In [ ]:
from airfoil.cnc import CNC
from airfoil.util.linestring_helpers import (
    split_and_roll,
    resample_linear_to_segment_length,
)
from airfoil.util.shapely_helpers import (
    plot_shapely_directional
)
import numpy as np
import shapely as sh
import time

In [ ]:
circle = sh.Point(0,-20).buffer(3.7)
circle = np.array(circle.exterior.coords)
circle = sh.LineString(resample_linear_to_segment_length(
    split_and_roll(circle,np.argmax(circle[:,1])),
    desired_segment_length=1
))
plunge = sh.LineString([
    [0,  0],
    [0,-20]
])
exit = sh.LineString([
    circle.coords[0],
    [0,  0],
])

plot_shapely_directional([plunge, circle,exit],legend=["plunge","circle","exit"])

In [ ]:
from dataclasses import dataclass
@dataclass
class OP:
    move:np.ndarray
    current:float
    speed:int
    pause_after_ms:float         = 0
    pause_after_current:float = 0
ops = [
    OP(
        move = np.array(plunge.coords),
        current = 2.5,
        speed = 400,
        pause_after_ms = 500,
        pause_after_current = 1.0,
    ),
    OP(
        move = np.array(circle.coords),
        current = 1.8,
        speed = 200,
    ),
    OP(
        move = np.array(circle.coords),
        current = 1.8,
        speed = 300,
        pause_after_ms = 50,
        pause_after_current = 0,
    ),
    OP(
        move = np.array(exit.coords),
        current = 2.5,
        speed = 400,
        pause_after_ms = 500,
        pause_after_current = 2.0,
    ),
    OP(
        move = np.array([]),
        current = 0,
        speed = 300,
        pause_after_ms = 0,
        pause_after_current = 0,
    ),
]

In [ ]:
def current_to_spindle(current:float):
    return max(0,min(4.0,current))/4 *80 + 10


In [ ]:
gcode:list[str] = []
for op in ops:
    gcode.append(f"M3 S{current_to_spindle(op.current):.2f}")
    speed = op.speed*np.sqrt(2)
    for x,y in op.move:
        gcode.append(f"G1 F{speed:.0f} X{x:.2f} Y{y:.2f} Z{x:.2f} A{y:.2f}")
    if op.pause_after_ms>0:
        gcode.append(f"M3 S{current_to_spindle(op.pause_after_current):.2f}")
        gcode.append(f"G4 P{op.pause_after_ms/1000:.3f}")
gcode.append(f"M3 S0")
with open("ffsake.gcode", "w", encoding="ascii") as f:
    f.write("\r\n".join(["G90",]+gcode)+"\r\n")
gcode

In [ ]:
cnc = CNC()

In [ ]:
cnc.alarm_soft_reset()
cnc.alarm_clear()

In [ ]:
cnc.home()
cnc.set_position(0,0,0,0)

In [ ]:
cnc.relative()
cnc.travel(
    100,180,
    100,180,
)

In [ ]:
cnc.absolute()
cnc.travel(0,0,0,0)

In [ ]:
cnc.read_all()

In [ ]:
cnc.relative()
cnc.travel(0,0,0,0)

In [ ]:
cnc.absolute()
cnc.set_position(0,0,0,0)
result = cnc.send_gcode_lines(gcode)

In [ ]:
print(result)

In [ ]:
cnc.relative()

cnc.feed(100,0,20,0,20)
cnc.serial.write("M3 S0\r\n".encode("ascii"))

In [ ]:
cnc.serial.write("M3 S0\r\n".encode("ascii"))

In [ ]:
#cnc.alarm_soft_reset()
cnc.alarm_clear()

In [ ]:
output

In [ ]:
cnc.status()